### 라이브러리 로드

In [0]:
!pip3 install pyLDAvis

     |████████████████████████████████| 1.6MB 4.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [0]:
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import re
from string import digits
import nltk; nltk.download('stopwords')
import os
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
# 파일 읽어오기
import pandas as pd
hotel_review = pd.read_csv('Hotel_Reviews.csv')
hotel_review.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397.0,1403.0,Only the park outside of the hotel was beauti...,11.0,7.0,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0.0,1403.0,No real complaints the hotel was great great ...,105.0,7.0,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42.0,1403.0,Location was good and staff were ok It is cut...,21.0,9.0,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210.0,1403.0,Great location in nice surroundings the bar a...,26.0,1.0,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140.0,1403.0,Amazing location and building Romantic setting,8.0,3.0,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [0]:
data1 = hotel_review.Negative_Review.values.tolist()
data1 = [re.sub("\'", "", str(sent)) for sent in data1]
data1 = [re.sub("No Negative'", "", str(sent)) for sent in data1]
pprint(data1[:1])

[' I am so angry that i made this post available via all possible sites i use '
 'when planing my trips so no one will make the mistake of booking this place '
 'I made my booking via booking com We stayed for 6 nights in this hotel from '
 '11 to 17 July Upon arrival we were placed in a small room on the 2nd floor '
 'of the hotel It turned out that this was not the room we booked I had '
 'specially reserved the 2 level duplex room so that we would have a big '
 'windows and high ceilings The room itself was ok if you don t mind the '
 'broken window that can not be closed hello rain and a mini fridge that '
 'contained some sort of a bio weapon at least i guessed so by the smell of it '
 'I intimately asked to change the room and after explaining 2 times that i '
 'booked a duplex btw it costs the same as a simple double but got way more '
 'volume due to the high ceiling was offered a room but only the next day SO i '
 'had to check out the next day before 11 o clock in order to ge

In [0]:
data2 = hotel_review.Positive_Review.values.tolist()
data2 = [re.sub("\'", "",str(t)) for t in data2]
data2 = [re.sub("No Positive", "", str(t)) for t in data2]
pprint(data2[:10])

[' Only the park outside of the hotel was beautiful ',
 ' No real complaints the hotel was great great location surroundings rooms '
 'amenities and service Two recommendations however firstly the staff upon '
 'check in are very confusing regarding deposit payments and the staff offer '
 'you upon checkout to refund your original payment and you can make a new one '
 'Bit confusing Secondly the on site restaurant is a bit lacking very well '
 'thought out and excellent quality food for anyone of a vegetarian or vegan '
 'background but even a wrap or toasted sandwich option would be great Aside '
 'from those minor minor things fantastic spot and will be back when i return '
 'to Amsterdam ',
 ' Location was good and staff were ok It is cute hotel the breakfast range is '
 'nice Will go back ',
 ' Great location in nice surroundings the bar and restaurant are nice and '
 'have a lovely outdoor area The building also has quite some character ',
 ' Amazing location and building Romantic

In [0]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words1 = list(sent_to_words(data1))
data_words2 = list(sent_to_words(data2))

print(data_words1[:1])
print(data_words2[:1])

[['am', 'so', 'angry', 'that', 'made', 'this', 'post', 'available', 'via', 'all', 'possible', 'sites', 'use', 'when', 'planing', 'my', 'trips', 'so', 'no', 'one', 'will', 'make', 'the', 'mistake', 'of', 'booking', 'this', 'place', 'made', 'my', 'booking', 'via', 'booking', 'com', 'we', 'stayed', 'for', 'nights', 'in', 'this', 'hotel', 'from', 'to', 'july', 'upon', 'arrival', 'we', 'were', 'placed', 'in', 'small', 'room', 'on', 'the', 'nd', 'floor', 'of', 'the', 'hotel', 'it', 'turned', 'out', 'that', 'this', 'was', 'not', 'the', 'room', 'we', 'booked', 'had', 'specially', 'reserved', 'the', 'level', 'duplex', 'room', 'so', 'that', 'we', 'would', 'have', 'big', 'windows', 'and', 'high', 'ceilings', 'the', 'room', 'itself', 'was', 'ok', 'if', 'you', 'don', 'mind', 'the', 'broken', 'window', 'that', 'can', 'not', 'be', 'closed', 'hello', 'rain', 'and', 'mini', 'fridge', 'that', 'contained', 'some', 'sort', 'of', 'bio', 'weapon', 'at', 'least', 'guessed', 'so', 'by', 'the', 'smell', 'of', 

In [0]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [0]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Remove Stop Words
data_words_nostops1 = remove_stopwords(data_words1)
data_words_nostops1 = remove_stopwords(data_words2)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized1 = lemmatization(data_words_nostops1, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized2 = lemmatization(data_words_nostops2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized1[:1])

[['park', 'outside', 'hotel', 'beautiful']]


In [0]:
# Create Dictionary
id2word1 = corpora.Dictionary(data_lemmatized1)
id2word2 = corpora.Dictionary(data_lemmatized2)

# Create Corpus
texts1 = data_lemmatized1
texts2 = data_lemmatized2

# Term Document Frequency
corpus1 = [id2word1.doc2bow(text1) for text1 in texts1]
corpus2 = [id2word2.doc2bow(text2) for text2 in texts2]

# View
print(corpus1[:1])
print(corpus2[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1)]]
[[(0, 7), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 15), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 5), (20, 1), (21, 1), (22, 1), (23, 3), (24, 2), (25, 2), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 3), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 3), (51, 1), (52, 2), (53, 1), (54, 6), (55, 1), (56, 1), (57, 1), (58, 2), (59, 1), (60, 3), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 5), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 3), (76, 1), (77, 1), (78, 6), (79, 2), (80, 1), (81, 1), (82, 2), (83, 2), (84, 2), (85, 1), (86, 3), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 13), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 2), (101, 1), (102, 7), (103, 1), (104, 1), (105, 1), (106, 1), (

In [0]:
id2word1[0]

'beautiful'

In [0]:
[[(id2word1[id], freq) for id, freq in cp1] for cp1 in corpus1[:1]]
[[(id2word2[id], freq) for id, freq in cp2] for cp2 in corpus2[:1]]

[[('-PRON-', 7),
  ('able', 1),
  ('again', 1),
  ('am', 1),
  ('angry', 1),
  ('annoying', 1),
  ('arrival', 1),
  ('as', 2),
  ('ask', 1),
  ('asleep', 1),
  ('attach', 1),
  ('audio', 1),
  ('available', 1),
  ('be', 15),
  ('because', 1),
  ('begin', 1),
  ('belonging', 1),
  ('big', 2),
  ('bio', 1),
  ('book', 5),
  ('booking', 1),
  ('break', 1),
  ('btw', 1),
  ('can', 3),
  ('ceiling', 2),
  ('change', 2),
  ('check', 2),
  ('city', 1),
  ('clock', 1),
  ('close', 1),
  ('com', 1),
  ('come', 1),
  ('constant', 2),
  ('contain', 1),
  ('cost', 1),
  ('day', 3),
  ('determine', 1),
  ('disturbing', 1),
  ('do', 1),
  ('don', 1),
  ('double', 1),
  ('duplex', 2),
  ('even', 1),
  ('evening', 1),
  ('explain', 1),
  ('fall', 1),
  ('floor', 1),
  ('fridge', 1),
  ('fully', 1),
  ('garden', 1),
  ('get', 3),
  ('good', 1),
  ('guess', 2),
  ('hard', 1),
  ('have', 6),
  ('hell', 1),
  ('hello', 1),
  ('here', 1),
  ('high', 2),
  ('holiday', 1),
  ('hotel', 3),
  ('internal', 1),


In [0]:
lda_model1 = gensim.models.ldamodel.LdaModel(corpus=corpus1,
                                           id2word=id2word1,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [0]:
lda_model2 = gensim.models.ldamodel.LdaModel(corpus=corpus2,
                                           id2word=id2word2,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [0]:
pprint(lda_model1.print_topics())
doc_lda1 = lda_model1[corpus1]

[(0,
  '0.170*"stay" + 0.071*"would" + 0.048*"night" + 0.046*"time" + '
  '0.032*"little" + 0.030*"book" + 0.028*"even" + 0.026*"find" + 0.024*"high" '
  '+ 0.023*"top"'),
 (1,
  '0.209*"really" + 0.168*"comfy" + 0.105*"definitely" + 0.100*"extremely" + '
  '0.100*"central" + 0.021*"morning" + 0.020*"choose" + 0.017*"second" + '
  '0.015*"immaculate" + 0.015*"order"'),
 (2,
  '0.499*"great" + 0.071*"place" + 0.041*"next" + 0.037*"lot" + 0.029*"brick" '
  '+ 0.028*"lane" + 0.016*"delicious" + 0.014*"subway" + 0.014*"luxurious" + '
  '0.012*"decent"'),
 (3,
  '0.400*"location" + 0.125*"excellent" + 0.059*"quiet" + 0.058*"perfect" + '
  '0.050*"london" + 0.027*"easy" + 0.024*"street" + 0.023*"east" + '
  '0.013*"superb" + 0.011*"access"'),
 (4,
  '0.154*"well" + 0.104*"small" + 0.088*"value" + 0.081*"city" + 0.053*"money" '
  '+ 0.048*"locate" + 0.038*"cleanliness" + 0.030*"kind" + 0.029*"work" + '
  '0.021*"museum"'),
 (5,
  '0.084*"paris" + 0.058*"go" + 0.050*"tower" + 0.045*"back" + 0.

In [0]:
print('\nPerplexity: ', lda_model1.log_perplexity(corpus1))
print('\nPerplexity: ', lda_model2.log_perplexity(corpus2))# a measure of how good the model is. lower the better.

# Compute Coherence Score(높을 수록 토픽 모델링이 잘되었다는 뜻.)
coherence_model_lda1 = CoherenceModel(model=lda_model1, texts=data_lemmatized1, dictionary=id2word1, coherence='c_v')
coherence_lda1 = coherence_model_lda1.get_coherence()
print('\nCoherence Score: ', coherence_lda1)
coherence_model_lda2 = CoherenceModel(model=lda_model2, texts=data_lemmatized2, dictionary=id2word2, coherence='c_v')
coherence_lda2 = coherence_model_lda2.get_coherence()
print('\nCoherence Score: ', coherence_lda2)


Perplexity:  -6.825078685254038

Perplexity:  -6.878956256264718

Coherence Score:  0.44442896036469115

Coherence Score:  0.4678144184613185


In [0]:
pyLDAvis.enable_notebook()
vis1 = pyLDAvis.gensim.prepare(lda_model1, corpus1, id2word1)
vis1

# 토픽은 1. service, 2. room, 3. location, 4. travel(transport)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.256357  0.306310       1        1  12.448574
8     -0.275391 -0.359067       2        1  11.633009
3     -0.174875  0.052446       3        1   9.968024
18    -0.146043  0.051431       4        1   8.063836
0      0.005823 -0.003315       5        1   5.395092
6     -0.000363 -0.000793       6        1   4.957603
7      0.036940 -0.003246       7        1   4.722137
5      0.045362 -0.002887       8        1   4.394522
2      0.016853  0.007434       9        1   4.201322
10     0.031520 -0.003476      10        1   3.848805
16     0.068083 -0.007097      11        1   3.511128
9      0.063435  0.000798      12        1   3.390059
4      0.035527 -0.002645      13        1   3.338111
14     0.072452  0.001369      14        1   3.159892
11     0.063423 -0.004538      15        1   3.046980
15     0.082884 -0.002065      16        1   2.937191
13     0.068250 -0.007597      17        1   2.900268
12     0.089335 -0.006309      18        1   2.899662
1      0.085593 -0.006514      19        1   2.619746
19     0.087548 -0.010238      20        1   2.564038, topic_info=     Category          Freq           Term         Total  loglift  logprob
24    Default  14691.000000       location  14691.000000  30.0000  30.0000
45    Default  15076.000000          staff  15076.000000  29.0000  29.0000
39    Default  13927.000000           room  13927.000000  28.0000  28.0000
58    Default   9121.000000           good   9121.000000  27.0000  27.0000
21    Default   7722.000000          great   7722.000000  26.0000  26.0000
1     Default  10426.000000          hotel  10426.000000  25.0000  25.0000
55    Default   5571.000000      breakfast   5571.000000  24.0000  24.0000
85    Default   6867.000000       friendly   6867.000000  23.0000  23.0000
290   Default   6003.000000          clean   6003.000000  22.0000  22.0000
177   Default   5700.000000        helpful   5700.000000  21.0000  21.0000
59    Default   5162.000000           nice   5162.000000  20.0000  20.0000
17    Default   4576.000000      excellent   4576.000000  19.0000  19.0000
269   Default   3378.000000           stay   3378.000000  18.0000  18.0000
130   Default   4548.000000    comfortable   4548.000000  17.0000  17.0000
89    Default   4297.000000            bed   4297.000000  16.0000  16.0000
118   Default   2018.000000         really   2018.000000  15.0000  15.0000
151   Default   2578.000000          close   2578.000000  14.0000  14.0000
52    Default   1887.000000           well   1887.000000  13.0000  13.0000
268   Default   2534.000000        station   2534.000000  12.0000  12.0000
76    Default   1823.000000         modern   1823.000000  11.0000  11.0000
184   Default   1792.000000       bathroom   1792.000000  10.0000  10.0000
127   Default   1620.000000          comfy   1620.000000   9.0000   9.0000
66    Default   2798.000000         lovely   2798.000000   8.0000   8.0000
68    Default   1790.000000        amazing   1790.000000   7.0000   7.0000
42    Default   1707.000000        service   1707.000000   6.0000   6.0000
62    Default   1433.000000           area   1433.000000   5.0000   5.0000
179   Default   1416.000000           love   1416.000000   4.0000   4.0000
98    Default   1985.000000           walk   1985.000000   3.0000   3.0000
82    Default   2152.000000          quiet   2152.000000   2.0000   2.0000
277   Default   2137.000000        perfect   2137.000000   1.0000   1.0000
...       ...           ...            ...           ...      ...      ...
27    Topic20    895.229126            new    896.120789   3.6626  -2.3553
482   Topic20    878.128662           free    879.020325   3.6626  -2.3746
1652  Topic20    603.424255        machine    604.315918   3.6621  -2.7498
241   Topic20    572.903320           give    573.794983   3.6620  -2.8017
736   Topic20    286.147491          suite    287.039154   3.6605  -3

In [0]:
pyLDAvis.enable_notebook()
vis2 = pyLDAvis.gensim.prepare(lda_model2, corpus2, id2word2)
vis2

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.415240 -0.243706       1        1  38.235195
1      0.211217  0.407085       2        1   9.979774
12     0.033147  0.015197       3        1   5.264517
7      0.008405  0.008761       4        1   4.426750
0      0.017156  0.009550       5        1   3.891998
18    -0.009918 -0.001039       6        1   3.777491
11     0.017167  0.011160       7        1   3.575520
10    -0.037329 -0.016791       8        1   3.333960
4     -0.014649 -0.000295       9        1   3.204819
19    -0.031080 -0.006903      10        1   3.056709
5     -0.043512 -0.011563      11        1   2.838672
8     -0.046642 -0.014758      12        1   2.499166
13     0.005948  0.004713      13        1   2.183776
14    -0.070181 -0.021413      14        1   2.142271
3     -0.070481 -0.021136      15        1   2.140366
6     -0.074173 -0.021963      16        1   1.952889
16    -0.067054 -0.021145      17        1   1.942112
15    -0.074232 -0.023266      18        1   1.925952
17    -0.083583 -0.025069      19        1   1.835820
9     -0.085445 -0.027420      20        1   1.792241, topic_info=     Category          Freq        Term         Total  loglift  logprob
93    Default  17253.000000        room  17253.000000  30.0000  30.0000
13    Default  39075.000000          be  39075.000000  29.0000  29.0000
135   Default   8973.000000    negative   8973.000000  28.0000  28.0000
306   Default   5050.000000   breakfast   5050.000000  27.0000  27.0000
151   Default   5973.000000        very   5973.000000  26.0000  26.0000
100   Default   5391.000000       small   5391.000000  25.0000  25.0000
280   Default   3417.000000     nothing   3417.000000  24.0000  24.0000
54    Default  10074.000000        have  10074.000000  23.0000  23.0000
78    Default   9824.000000         not   9824.000000  22.0000  22.0000
76    Default   2362.000000       night   2362.000000  21.0000  21.0000
214   Default   2329.000000         too   2329.000000  20.0000  20.0000
159   Default   3077.000000         bed   3077.000000  19.0000  19.0000
0     Default   7341.000000      -PRON-   7341.000000  18.0000  18.0000
626   Default   1651.000000     service   1651.000000  17.0000  17.0000
210   Default   1721.000000      shower   1721.000000  16.0000  16.0000
60    Default   6009.000000       hotel   6009.000000  15.0000  15.0000
249   Default   1594.000000      really   1594.000000  14.0000  14.0000
136   Default   1379.000000         bar   1379.000000  13.0000  13.0000
72    Default   1632.000000        more   1632.000000  12.0000  12.0000
465   Default   1676.000000      little   1676.000000  11.0000  11.0000
198   Default   1489.000000       price   1489.000000  10.0000  10.0000
172   Default   1384.000000        door   1384.000000   9.0000   9.0000
218   Default   1326.000000       water   1326.000000   8.0000   8.0000
138   Default   2015.000000         bit   2015.000000   7.0000   7.0000
530   Default   1200.000000        poor   1200.000000   6.0000   6.0000
329   Default   1981.000000    bathroom   1981.000000   5.0000   5.0000
529   Default   1132.000000       quite   1132.000000   4.0000   4.0000
192   Default   1209.000000       noisy   1209.000000   3.0000   3.0000
23    Default   1324.000000         can   1324.000000   2.0000   2.0000
46    Default   1278.000000       floor   1278.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
626   Topic20   1650.329102     service   1651.344971   4.0211  -1.6072
431   Topic20   1082.015137     morning   1083.031006   4.0208  -2.0293
274   Topic20    828.305237  everything    829.321167   4.0205  -2.2965
2137  Topic20    174.496231        meal    175.512146   4.0159  -3.8540
1838  Topic20    172.310806  absolutely    173.326721   4.0158  -3.8666
1770  Topic20    143.672073      dining    144.687988   4.0147  -4.0484
623   Topic20    136.956299      